In [ ]:
from bs4 import BeautifulSoup 
import pandas as pd
import requests 
import re

pd.set_option ('display.max_columns', None)

In [ ]:
url = "https://moviesdatabase.p.rapidapi.com/titles"

headers = {
    "X-RapidAPI-Key": "431e779057msh1f0c87c2937a329p125efcjsn45064e4b4925",
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

querystring = {
    "genre": "Drama",
    "startYear": "2010",
    "titleType": "movie",
    "endYear": "2024"
}

def llamar_pelis(url, headers, querystring):
    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error en la petición tipo:", response.status_code, response.reason)
        return None


In [ ]:
llamar_pelis (url, headers, querystring)

In [ ]:
# Llamada inicial a la API
resultados_iniciales = llamar_pelis(url, headers, querystring)
if resultados_iniciales:
    print(resultados_iniciales)  # Imprime los resultados iniciales para verificar
else:
    print("No se recibieron datos.")

In [ ]:
def llamar_pelis(url, headers, querystring):
    datos_paginas = []
    base_url = "https://moviesdatabase.p.rapidapi.com"  # url base para completar las urls de las paginas
    while url:
        # Verifica si la url esta completa, si no, la completa con nuestros parametros. Evita errores
        if not url.startswith('http'):
            url = base_url + url
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            json_pelis = response.json()
            datos_paginas.extend(json_pelis['results'])  
            url = json_pelis.get('next')  # Obtiene la siguiente página si existe
            querystring = {}  # Limpia el querystring porque la url 'next' debería incluir los paráme
        else:
            print("Error en la solicitud:", response.status_code, response.reason)
            return None
    return {"results": datos_paginas}

In [ ]:
# Llamada completa con paginación
datos_limpios = llamar_pelis(url, headers, querystring)
if datos_limpios:
    print(datos_limpios)  # Imprime los resultados para verificar la paginación
else:
    print("Opsss, parece que no se han recibido datos")

In [ ]:
def limpiar_json(datos_json):
    # Lista para mantener los registros extraídos
    peliculas = []

    # Itera sobre cada película en los resultados
    for pelicula in datos_json['results']:
        # Extrae los datos requeridos
        tipo = pelicula['titleType']['text']
        nombre = pelicula['titleText']['text']
        year = pelicula['releaseYear']['year']
        mes = pelicula['releaseDate']['month'] if 'releaseDate' in pelicula and pelicula['releaseDate'] is not None else 'Desconocido'
        id_pelicula = pelicula['id']

        # Agrega un registro por cada película
        peliculas.append({
            'Tipo': tipo,
            'Nombre': nombre,
            'Año de Estreno': year,
            'Mes de Estreno': mes,
            'ID': id_pelicula
        })

    # Crea un DataFrame con la lista de registros
    dataframe_peliculas = pd.DataFrame(peliculas)
    
    return dataframe_peliculas

In [ ]:
datos_limpios = llamar_pelis(url, headers, querystring)
df = limpiar_json(datos_limpios)

In [ ]:
df